In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
loader = PyPDFLoader("./book_1.pdf")
docs = loader.load()


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks = splitter.split_documents(docs)

In [ ]:
map_prompt = PromptTemplate(
    template = """
Write a concise summary of the following text. Keep only the important ideas.

{text}
"""
, input_variables=["text"]
)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
parser = StrOutputParser()

chain = map_prompt | llm | parser

chunk_summaries  = [chain.invoke({"text": chunk.page_content}) for chunk in chunks]


In [ ]:
reduce_prompt = PromptTemplate(
    template = """
Combine the following chunk summaries into one clear, concise, well-structured summary:

{text}

Final Summary:
""",
    input_variables=["text"]
)

In [ ]:
last_chain = reduce_prompt | llm | parser

combined_summaries = "\n\n---\n\n".join(chunk_summaries)

result = last_chain.invoke({"text": combined_summaries})

print(result)